In [1]:
#!usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
import tensorflow as tf
import numpy as np
import const as C

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# filter = [kernel_height, kernel_width, input_channel, output_channel]
# 畳み込み層(エンコーダ部分)
def conv2d(x, output_channel, kernel=3, stride=2, batch_norm=True, is_training=True, leaky_relu=True):
    net = tf.layers.conv2d(x,
                          filters=output_channel,
                          kernel_size=[kernel, kernel],
                          strides=[stride, stride],
                          padding='SAME')
    if batch_norm:
        net = tf.layers.batch_normalization(net, training=is_training)
    if leaky_relu:
        net = tf.nn.leaky_relu(net, 0.2)
    return net

In [4]:
# filter = [kernel_height, kernel_width, output_channel, input_channel]
# output_shape = [バッチ数, 得たいheight, 得たいwidth, 得たいchannel]
# 逆畳み込み層(デコーダ部分)
def de_conv2d(x, output_channel, kernel=3, stride=2, batch_norm=True, is_training=True, relu=True):
    net = tf.layers.conv2d_transpose(x,
                                     filters=output_channel,
                                     kernel_size=[kernel, kernel],
                                     strides=[stride, stride],
                                     padding='SAME')
    if batch_norm:
        net = tf.layers.batch_normalization(net, training=is_training)
    if relu:
        net = tf.nn.relu(net)
    return net

In [5]:
def UNet():
    net = {}
    net['input'] = tf.placeholder(tf.float32, [None, 512, 128, 1])
    net['conv1'] = conv2d(net['input'], output_channel=16)
    net['conv2'] = conv2d(net['conv1'], output_channel=32)
    net['conv3'] = conv2d(net['conv2'], output_channel=64)
    net['conv4'] = conv2d(net['conv3'], output_channel=128)
    net['conv5'] = conv2d(net['conv4'], output_channel=256)
    net['conv6'] = conv2d(net['conv5'], output_channel=512)
    net['de_conv1'] = de_conv2d(net['conv6'], output_channel=256)
    net['concat1'] = tf.concat([net['de_conv1'], net['conv5']], axis=-1)
    net['dropout1'] = tf.nn.dropout(net['concat1'], rate=0.5)
    net['de_conv2'] = de_conv2d(net['dropout1'], output_channel=128)
    net['concat2'] = tf.concat([net['de_conv2'], net['conv4']], axis=-1)
    net['dropout2'] = tf.nn.dropout(net['concat2'], rate=0.5)
    net['de_conv3'] = de_conv2d(net['dropout2'], output_channel=64)
    net['concat3'] = tf.concat([net['de_conv3'], net['conv3']], axis=-1)
    net['dropout3'] = tf.nn.dropout(net['concat3'], rate=0.5)
    net['de_conv4'] = de_conv2d(net['dropout3'], output_channel=32)
    net['concat4'] = tf.concat([net['de_conv4'], net['conv2']], axis=-1)
    net['de_conv5'] = de_conv2d(net['concat4'], output_channel=16)
    net['concat5'] = tf.concat([net['de_conv5'], net['conv1']], axis=-1)
    net['de_conv6'] = de_conv2d(net['concat5'], output_channel=1)
    return net

In [6]:
model = UNet()

# 初期化
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
